# Fitting Eulers Equations

$$
    \tau = J \dot{\omega} + \omega \times (J \omega) \\
    \\
    \tau = I_3 J \dot{\omega} + \hat{\omega} J \omega \\
    \\
    \tau = (\dot{\omega}^T \otimes I_3) \text{vec}(J) + (\omega^T \otimes \hat{\omega}) \text{vec}(J) \\
    \\
    \tau = (\dot{\omega}^T \otimes I_3 + \omega^T \otimes \hat{\omega}) \text{vec}(J) \\
$$

The model input at time step $k$ is then $y_k$
$$
    y_k =
    \left[\begin{array}{cccc}
        v_B^T & \omega_B^T & \Omega^T & \dot{\Omega}^T \\
    \end{array}\right] ^ T\\
    y_k \in \mathbb{R}^{14}\\
$$

The matrix of coefficients is then defined as $C \in \mathbb{R}^{6 \times 13}$. The first three block terms in $x$ are velocities and are used for modeling aerodynamic forces/torques, while the fourth term is an acceleration which is used for modeling momentum conservation moments.

In [21]:
x = rand(3 + 3 + 4 + 4)
size([1; x; kron(x, x)])

(211,)

$$
    z_k = \left[\begin{array}{c}
        1 \\
        y_k \\
        y_k \otimes y_k \\
    \end{array}\right] \in \mathbb{R}^{183}
$$
$$
    \hat{\tau} = C \, z_k \\
    C \in \mathbb{R}^{ 3 \times 183 }
$$

$$
    \text{vec}(S) = A \, \text{vech}(S) : \forall \, S \in Sym( \mathbb{R^{3 \times 3}}) \\
    A \in \mathbb{R^{3 \times 3}}, \,
    A = \left[\begin{array}{cccccc}
        1 & 0 & 0 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 1 & 0 & 0 \\
        0 & 1 & 0 & 0 & 0 & 0 \\
        0 & 0 & 1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 0 & 0 & 1 \\
    \end{array}\right]
$$

$$
    \begin{aligned}
        \tau_k - \hat{\tau}_k &= \underbrace{(\dot{\omega}_k^T \otimes I + \omega_k^T \otimes \hat{\omega}_k) \, A }_{B \in \mathbb{R}^{3 \times 6}} \, \underbrace{\text{vech}(J)}_{j} - C \, z_k \\

        &= B_k \, \text{vech}(J) - C \, z_k\\
    \end{aligned}
$$

$$
    \begin{aligned}

        \tau_k - \hat{\tau}_k &= \underbrace{(\dot{\omega}_k^T \otimes I + \omega_k^T \otimes \hat{\omega}_k) \, A }_{B \in \mathbb{R}^{3 \times 6}} \, \underbrace{\text{vech}(J)}_{j} - C \, z_k \\

        &= I_3 \, B \, j - C \, z\\

        &= (j^T \otimes I_3) \text{vec}(B_k) - C \, z_k\\

        &= \left[\begin{array}{cc} (j^T \otimes I_3) & - C \end{array}\right] 
        
        \left[\begin{array}{c} 
            \text{vec}(B_k) \\
            z_k 
        \end{array}\right]
    \end{aligned}
$$

$$
    \begin{aligned}
        \tau_k - \hat{\tau}_k =& 
        \underbrace{\left[\begin{array}{cc} (\text{vech}(J)^T \otimes I_3) & - C \end{array}\right]}_{D}
        \underbrace{\left[\begin{array}{c} 
            \text{vec}\left(\left(\dot{\omega}_k^T \otimes I + \omega_k^T \otimes \hat{\omega}_k\right)\, A\right)   \\
            z_k
        \end{array}\right]}_{x_k}\\

    \end{aligned}
$$

$$
    \begin{aligned}
        \min_{D} \sum_k \lVert D \, x_k \rVert ^2

    \end{aligned}
$$

Constraint on the first block term

In [2]:
using LinearAlgebra
using Symbolics
using Latexify
using CSV
using DataFrames

In [10]:
hat(x) = [0. -x[3] x[2]; x[3] 0. -x[1]; -x[2] x[1] 0.]

function vech(x)
    k = size(x, 1)
    a = zeros(eltype(x), Int((k^2-k)/2 + k))
    m = 1
    for i = 1:k
        for j = 1:i
            a[m] = x[i,j]
            m += 1
        end
    end
    a
end

A = [1 0 0 0 0 0;
     0 1 0 0 0 0;
     0 0 0 1 0 0;
     0 1 0 0 0 0;
     0 0 1 0 0 0;
     0 0 0 0 1 0;
     0 0 0 1 0 0;
     0 0 0 0 1 0;
     0 0 0 0 0 1];


# @variables ω[1:3] ω̇[1:3] J[1:3, 1:3]
ω = rand(3)
ω̇ = rand(3)

(kron(ω̇', I(3)) + kron(ω', hat(ω))) * A

3×6 Matrix{Float64}:
  0.152073  0.712965  -0.0966919   1.13674    -0.108293   0.0966919
  0.24326   0.248765   0.956225   -0.193743    0.851575  -0.24326
 -0.142581  0.302036   0.142581    0.0553808   1.19948    0.994156

In [8]:
A

9×6 Matrix{Int64}:
 1  0  0  0  0  0
 0  1  0  0  0  0
 0  0  0  1  0  0
 0  1  0  0  0  0
 0  0  1  0  0  0
 0  0  0  0  1  0
 0  0  0  1  0  0
 0  0  0  0  1  0
 0  0  0  0  0  1

In [10]:
@variables u v w x y z

test = [u v w; 
        v x y;
        w y z];
A = [1 0 0 0 0 0;
     0 1 0 0 0 0;
     0 0 0 1 0 0;
     0 1 0 0 0 0;
     0 0 1 0 0 0;
     0 0 0 0 1 0;
     0 0 0 1 0 0;
     0 0 0 0 1 0;
     0 0 0 0 0 1];
b = vech(test)

A * vech(test) 

9-element Vector{Num}:
 u
 v
 w
 v
 x
 y
 w
 y
 z

In [63]:
df = CSV.read("processed_data/merged_2021-02-03-16-55-55_seg_2.csv", DataFrame)
# names(df)

p = Matrix(df[!, ["pos x","pos y","pos z"]])
v = Matrix(df[!, ["vel x","vel y","vel z"]])
v̇ = Matrix(df[!, ["acc x","acc y","acc z"]])
ω = Matrix(df[!, ["ang vel x","ang vel y","ang vel z"]])
ω̇ = Matrix(df[!, ["ang acc x","ang acc y","ang acc z"]])
Ω = Matrix(df[!, ["mot 1","mot 2","mot 3","mot 4"]])
Ω̇ = Matrix(df[!, ["dmot 1","dmot 2","dmot 3","dmot 4"]]);

𝕏𝑓 = hcat(v, ω, Ω);
𝕏𝑡 = hcat(v, ω, ω̇, Ω, Ω̇);

size(𝕏𝑡)


(12363, 17)

In [ ]:
function residual(
    x::MVector{17},
    β::MVector{93},
)
    v, ω, ω̇, Ω, Ω̇ = x[1:3], x[4:6], x[7:9], x[10:13], x[14:17]

    # D ∈ ℝ³ˣ²¹¹ 
    D = reshape(β[1:87], 3, 29)
    J₁₁, J₁₂, J₁₃, J₂₂, J₂₃, J₃₃ = β[end-5:end]
    𝕁 = [J₁₁  J₁₂  J₁₃;
         J₁₂  J₂₂  J₂₃;
         J₁₃  J₂₃  J₃₃];

    # Body torque
    τ = 𝕁 * ω̇ + ω × (𝕁 * ω)

    # Quadratic model
    y = [v; ω; Ω; Ω̇]
    # z = [1; y; kron(y, y)]
    z = [1; y; y.*y]

    τ̂ = D * z

    r = τ - τ̂
    return r
end